### Implement selection and sorting of electrons

In [1]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# %matplotlib inline

In [121]:
import skimage.measure
from skimage.util import view_as_blocks
import logging

In [122]:
log = logging.getLogger(__name__)

### Importing the data

We are using the Merged Electron Region data.

In [2]:
event_data = pd.read_csv("output_data/merged_energy_electron_data.csv") # Electron Region Event one

In [3]:
# View the merged data head
event_data.head()

,event,eta,phi,et,position,electron,tau
0,1,0,0,7.23932,3,0,0
1,1,0,1,14.78990,6,0,1
2,1,0,2,8.17547,4,0,1
3,1,0,3,6.89318,2,0,0
4,1,0,4,9.89150,7,0,0


### ET values

In [7]:
# Convert the series into a 14x18 matrix
x_et = np.array(event_data['et'])
matrix_et = np.asmatrix(x_et)

# Resizing into size 14x18
matrix_et.resize((14, 18))

### Electron values

In [6]:
# Convert the series into a 14x18 matrix
x_elec = np.array(event_data['electron'])
matrix_electron = np.asmatrix(x_elec)

# Resizing into size 14x18
matrix_electron.resize((14, 18))

### Reduce the dimensions of the matrix using 2x2 regions

We will now reduce the dimensions of our matrix from 14x18 to 7x9 using 2x2 region replacements, i.e., replacing 2x2 regions by only the highest value of ET (energy) with true electrons.

In [ ]:
# Configuring the size of sub regions
s = 2

We will now get 2x2 sub matrices as blocks from the electron matrix.

In [119]:
blocks_elec = skimage.util.view_as_blocks(matrix_electron, (s, s))

Reshape each matrix into a list format.

In [120]:
blocks_elec = blocks_elec.reshape(-1,s**2)

In [153]:
# Initialize empty dictionary to save row and index of energy values in matrix
energy_dict = {}

# Loop inside sub matrix of blocks
for i, sub in enumerate(blocks_elec):
    count = 0 # initially count is 0
    
    # Loop inside each element of submatrix
    for idx, element in enumerate(sub):
        if element == 1: # if value of electron is 1
            log.info("Contains Electron in:", idx)
            if count < 1:
                # add to dictionary
                energy_dict[i] = [idx]
            else:
                # append to dictionary
                energy_dict[i].append(idx)
            # increase the count    
            count += 1
        else:
            # Contains no electron True (1)
            
            if count == 0:
                energy_dict[i] = ['NaN']
            log.info("No electron in row")
            
    # Check if count is greater than or equal to 1        
    if count >= 1:
        log.info("Energy is present in matrix:", i)
    else:
        log.info("No energy in matrix:", i)

In [154]:
energy_dict

{0: ['NaN'],
 1: ['NaN'],
 2: ['NaN'],
 3: ['NaN'],
 4: ['NaN'],
 5: ['NaN'],
 6: ['NaN'],
 7: ['NaN'],
 8: [2],
 9: ['NaN'],
 10: [1],
 11: [0],
 12: ['NaN'],
 13: [3],
 14: ['NaN'],
 15: [1],
 16: ['NaN'],
 17: ['NaN'],
 18: ['NaN'],
 19: [2],
 20: [1],
 21: ['NaN'],
 22: [2, 3],
 23: ['NaN'],
 24: [1, 2],
 25: ['NaN'],
 26: ['NaN'],
 27: [1],
 28: ['NaN'],
 29: [3],
 30: [3],
 31: [2],
 32: ['NaN'],
 33: [1],
 34: ['NaN'],
 35: ['NaN'],
 36: ['NaN'],
 37: ['NaN'],
 38: ['NaN'],
 39: ['NaN'],
 40: ['NaN'],
 41: [1, 3],
 42: ['NaN'],
 43: ['NaN'],
 44: [0],
 45: ['NaN'],
 46: ['NaN'],
 47: ['NaN'],
 48: ['NaN'],
 49: [0, 3],
 50: ['NaN'],
 51: ['NaN'],
 52: [3],
 53: ['NaN'],
 54: ['NaN'],
 55: [1, 2],
 56: ['NaN'],
 57: [0],
 58: ['NaN'],
 59: ['NaN'],
 60: ['NaN'],
 61: ['NaN'],
 62: [2]}

In [156]:
# Get list of matrix rows with electron True
matrices_with_energy = [k for k,v in energy_dict.items() if v != ['NaN']]

We will now get 2x2 sub matrices as blocks from the energy (et) matrix.

In [157]:
blocks_energy = view_as_blocks(matrix_et, (s,s)).reshape(-1,s**2)

In [158]:
final_energy = []

# Loop within submatrix of 2x2 energy value blocks
for row, sub in enumerate(blocks_energy):
    # Check if row contains electron True values
    if row in matrices_with_energy:
        log.info("Contains energy values in row:", row)
        
        # Get the value of index 
        index = np.argmax(sub[energy_dict[row]])
        
        # Append the tuple (max energy, index)
        final_energy.append((sub[energy_dict[row]].max(), energy_dict[row][index]))
    else:
        final_energy.append((0, energy_dict[row] ))

In [159]:
final_energy

[(0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (12.8206, 2),
 (0, ['NaN']),
 (0.0, 1),
 (16.15002, 0),
 (0, ['NaN']),
 (13.3519, 3),
 (0, ['NaN']),
 (64.3284, 1),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (8.14362, 2),
 (10.5057, 1),
 (0, ['NaN']),
 (13.4624, 2),
 (0, ['NaN']),
 (13.7001, 1),
 (0, ['NaN']),
 (0, ['NaN']),
 (11.288, 1),
 (0, ['NaN']),
 (12.3744, 3),
 (8.79125, 3),
 (9.65047, 2),
 (0, ['NaN']),
 (5.67157, 1),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (12.0682, 3),
 (0, ['NaN']),
 (0, ['NaN']),
 (9.62609, 0),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (11.2701, 0),
 (0, ['NaN']),
 (0, ['NaN']),
 (12.4474, 3),
 (0, ['NaN']),
 (0, ['NaN']),
 (15.3848, 1),
 (0, ['NaN']),
 (4.03492, 0),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (0, ['NaN']),
 (12.1941, 2)]

In [151]:
len(final_energy)

63

Save them into a final array.

Reshape them as 7x9

Save their index as well

sort them based on energy values into 6 electrons.

### Get 6 electrons

- Sort them based on their energies

To do: 
Get time complexity